In [3]:
from PIL import Image
import numpy as np
import cv2
import time
import sys

%run "common_functions.py"

sys.setrecursionlimit(53000) # override needed for computing midpoints, which uses a recursive function
Image.MAX_IMAGE_PIXELS = 366498276 # override is needed, or else it gives a DecompressionBombError

# cy1_shifted_file = "originals/shiftedcycle 1thnksgv.tif"
# cy2_shifted_file = "originals/shiftedcycle 2thnksgv.tif"

cy1_shifted_file = "shif_3x3.png"
cy2_shifted_file = "orig_3x3.png"

def c(img):
    return Image.fromarray(img)

def overlay(arr1, arr2):
    return (arr1 / 3 + arr2 / 2);

DIPlib -- a quantitative image analysis library
Version 3.4.1 (Oct 13 2023)
For more information see https://diplib.org


In [6]:


def alignImages(im1, im2):
  # Convert images to grayscale
  #im1Gray = cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY)
  #im2Gray = cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY)
 
  # Detect ORB features and compute descriptors.
  orb = cv2.ORB_create(10000)
  keypoints1, descriptors1 = orb.detectAndCompute(im1, None)
  keypoints2, descriptors2 = orb.detectAndCompute(im2, None)
 
  # Match features.
  matcher = cv2.DescriptorMatcher_create(cv2.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING)
  matches = list(matcher.match(descriptors1, descriptors2, None))
 
  # Sort matches by score
  matches.sort(key=lambda x: x.distance, reverse=False)
 
  # Remove not so good matches
  print("Count of Matches:", len(matches))
  numGoodMatches = int(len(matches) * .01)
  print("Count of Good Matches:", numGoodMatches)
  print("Distance of worst match:", matches[numGoodMatches].distance)
  matches = matches[:numGoodMatches]
 
  # Draw top matches
  #imMatches = cv2.drawMatches(im1, keypoints1, im2, keypoints2, matches, None)
  #cv2.imwrite("matches.jpg", imMatches)
 
  # Extract location of good matches
  points1 = np.zeros((len(matches), 2), dtype=np.float32)
  points2 = np.zeros((len(matches), 2), dtype=np.float32)
 
  for i, match in enumerate(matches):
    points1[i, :] = keypoints1[match.queryIdx].pt
    points2[i, :] = keypoints2[match.trainIdx].pt
 
  # Find homography
  h, mask = cv2.findHomography(points1, points2, cv2.RANSAC)
 
  # Use homography
  height, width = np.array(im2.shape) * 2
  im1Reg = cv2.warpPerspective(im1, h, (width, height))
  
  # now, find and print the x y shift that the center experiences... 
  disp = np.dot(h, [250, 250, 1])
  print((disp * disp[2])[0:2] - 250)
  
  xydisp = (disp * disp[2])[0:2] - 250
  
  
 
  return im1Reg, h, xydisp
 


In [7]:
layer_to_align = 2

img_cy1 = Image.open(cy1_shifted_file)
brightfield_cy1 = np.array(img_cy1)
if brightfield_cy1.dtype != "uint8":
    brightfield_cy1 = (brightfield_cy1/256).astype('uint8') # don't need the whole range of values, so this reduces mem size, + improves access speed

img_cy2 = Image.open(cy2_shifted_file)
brightfield_cy2 = np.array(img_cy2)
if brightfield_cy2.dtype != "uint8":
    brightfield_cy2 = (brightfield_cy2/256).astype('uint8') 

img_cy1 = brightfield_cy1
img_cy2 = brightfield_cy2

brightfield_cy1[brightfield_cy2 == 0] = 0
brightfield_cy2[brightfield_cy1 == 0] = 0

print(brightfield_cy1.shape)
brightfield_cy1 = brightfield_cy1[10000:12000, 10000:12000]
brightfield_cy2 = brightfield_cy2[10000:12000, 10000:12000]
print(brightfield_cy1.shape)

(15640, 15640)
(2000, 2000)


In [12]:
import pyelastix
import os 
os.environ["ELASTIX_PATH"] = "/Users/clark/Desktop/aresearchdemo/align_reg/elastix/bin/elastix"
# os.environ["LD_LIBRARY_PATH"] = "/Users/clark/Downloads/elastix-5.1.0-mac/bin/elastix"

# print(os.environ["ELASTIX_PATH"])

# Get params and change a few values
params = pyelastix.get_default_params()
params.MaximumNumberOfIterations = 200
params.FinalGridSpacingInVoxels = 10


c_bf_1 = np.ascontiguousarray(brightfield_cy1)
c_bf_2 = np.ascontiguousarray(brightfield_cy2)

# Apply the registration (im1 and im2 can be 2D or 3D)
im1_deformed, field = pyelastix.register(c_bf_1,c_bf_2, params)

c(brightfield_cy1[1000:1600, 1000:1600]).convert("RGB")

RuntimeError: Could not find Elastix executable. Download Elastix from http://elastix.isi.uu.nl/. Pyelastix looks for the exe in a series of common locations. Set ELASTIX_PATH if necessary.

In [ ]:
print(brightfield_cy1.flags)

In [ ]:
## next, find average shift

def blockify(cuts):
    
    # Returns the center points of all the blocks in a cuts x cuts array
    
    centerpoints = []
    for i in range(cuts):
        row = []
        for j in range(cuts):
            # print((i + 1), cuts, (j + 1), cuts)
            row.append(np.array([(2*i + 1) / (cuts *2), (2*j + 1) / (cuts *2)]))
            # print((2*i + 1) / (cuts *2))
        
        centerpoints.append(np.array(row))
            
    return np.array(centerpoints)



# https://factorization.info/factors/1/factors-of-17472.html 
# the second param as below can be
block_points = blockify(4) * brightfield_cy1.shape[0]
girth = int(block_points[0][0][0])

avg_disp = []

n = -3
for i in block_points:
    for j in i:
        if n == 1:
            break
        n += 1
        x,y = j.astype(int)
        alngd, h, xydisp = (alignImages(brightfield_cy1[y-girth:y+girth, x- girth:x+girth,], brightfield_cy2[y-girth:y+girth, x- girth:x+girth,]))
        avg_disp.append(xydisp)

print(avg_disp)
print(np.mean(avg_disp, axis=0))
x_shift,y_shift = np.mean(avg_disp, axis=0).astype(int)
c(img_cy2[10000+y_shift:12000+ y_shift, 10000+x_shift:12000+x_shift])
brightfield_cy2 = img_cy2[10000+y_shift:12000+ y_shift, 10000+x_shift:12000+x_shift]

In [ ]:
block_points = blockify(10) * brightfield_cy1.shape[0]
girth = int(block_points[0][0][0])

n = -222
new_crew = c(np.zeros((2000,2000))).convert("RGBA")
lol=[]
for i in block_points:
    for j in i:
        # x,y = j.astype(int)
        # lol = brightfield_cy1[y-girth:y+girth, x- girth:x+girth,]
        try:
            # break
            if n == 1:
                break
            n += 1
            x,y = j.astype(int)
            alngd, h, xy = (alignImages(adjust_contrast(brightfield_cy1[y-girth:y+girth, x- girth:x+girth,]), adjust_contrast(brightfield_cy2[y-girth:y+girth, x- girth:x+girth,])))
            arr = np.array((c(alngd)).convert("RGBA"))
            arr[:, :, 3] = (255 * (arr[:, :, :3] != 0).any(axis=2)).astype(np.uint8)
            print(x, y )
            
            new_crew.paste(c(arr),(x-girth, y-girth), c(arr))
        except:
            pass

new_crew
# # c(brightfield_cy2)
# c(lol).convert("RGBA")

In [ ]:
c(brightfield_cy1)

In [ ]:
# def blockify(cuts):
    
#     # Returns the center points of all the blocks in a cuts x cuts array
    
#     centerpoints = []
#     for i in range(cuts):
#         row = []
#         for j in range(cuts):
#             # print((i + 1), cuts, (j + 1), cuts)
#             row.append(np.array([(2*i + 1) / (cuts *2), (2*j + 1) / (cuts *2)]))
#             # print((2*i + 1) / (cuts *2))
        
#         centerpoints.append(np.array(row))
            
#     return np.array(centerpoints)



# # https://factorization.info/factors/1/factors-of-17472.html 
# # the second param as below can be
# block_points = blockify(4) * brightfield_cy1.shape[0]
# width = brightfield_cy1.shape[0] / 32 / 2
# # print( block_points )
# lol = brightfield_cy2.copy()
# for i in block_points:
#     for j in i:
#         # print(j)
#         j = j.astype(int)
        
#         lol = cv2.circle(lol, j.astype(int), 10, 255, 10)
#         x,y = j
#         girth = int(block_points[1][1][0])
#         cv2.rectangle(lol, (x-girth, y+girth), (x+girth, y-girth), (255, 255, 255), 2) 
#         # print((x-girth, y-girth), (x+girth, y+girth))
  

# x,y = block_points[1][2].astype(int)
# print(x,y)
# girth = int(block_points[0][0][0])
# print(girth)
# # print(x - girth, x+girth, y-girth, y+girth )    
# # print(slice(x-girth, y+girth), slice(x-girth, y+girth))
# cv2.rectangle(lol, (x-girth, y+girth), (x+girth, y-girth), (255, 255, 255), 102) 
# print((x-girth, y+girth), (x+girth, y-girth))
# c(lol)
# # LETS GOOOOOOO FIGURED IT OUUUTTTTT!!
# c(brightfield_cy1[y-girth:y+girth, x- girth:x+girth,])

# n = -222
# new_crew = c(np.zeros((2000,2000))).convert("RGBA")

# for i in block_points:
#     for j in i:
#         if n == 1:
#             break
#         n += 1
        
#         x,y = j.astype(int)
        
        
#         alngd, h, xydisp = (alignImages(brightfield_cy1[y-girth:y+girth, x- girth:x+girth,], brightfield_cy2[y-girth:y+girth, x- girth:x+girth,]))
#         print((x,y), np.add(j.astype(int), xydisp).astype(int))
#         x,y = np.add(j.astype(int), xydisp).astype(int)
        
#         bf1, bf2 = brightfield_cy1[y-girth:y+girth, x- girth:x+girth,], brightfield_cy2[y-girth:y+girth, x- girth:x+girth,]
        
#         cut_into_4 = blockify(2) * bf1.shape[0]
        
#         for row in cut_into_4:
#             for item in row:
#                 girth2 = int(cut_into_4[0][0][0])
#                 x2, y2 = item.astype(int)
                
#                 alngd, h, xydisp = (alignImages(bf1[y2-girth2:y2+girth2, x2 - girth2:x2 + girth2,], bf2[y2 - girth2:y2 + girth2, x2 - girth2:x2 + girth2,]))
                
                
#                 arr = np.array((c(alngd)).convert("RGBA"))
#                 arr[:, :, 3] = (255 * (arr[:, :, :3] != 0).any(axis=2)).astype(np.uint8)
#                 print(x, y )
                
#                 new_crew.paste(c(arr),(x-girth, y-girth), c(arr))

#         # new_crew[y-girth:y+girth, x- girth:x+girth,] = alngd

# new_crew
# # c(brightfield_cy2)
